In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
# renderer for jupyter notebook
pio.renderers.default='notebook'
# renderer for vs code
# pio.renderers.default='vscode'
# renderer for colab
# pio.renderers.default='colab'
pio.templates.default = "plotly_dark"
import numpy as np
import statsmodels.api as sm
import datetime
from plotly.subplots import make_subplots
from ipywidgets import widgets,Layout

In [2]:
rain=pd.read_csv(r'../data/processedfiles/rainfall-by-region.csv')
rain=rain.set_index('date',drop=False)
regions=list(rain.columns.values)[5:]
years=list(rain['year'].unique())

In [3]:
# rain considered from Day number 150 to 280

In [4]:
monsoon_list=[i for i in range(150,281)]
rain_monsoon=rain[rain['day-number'].isin(monsoon_list)]

### Plotting Functions

In [5]:
# dropdown widget
region = widgets.Dropdown(
    options=regions,
    value=regions[0], description='Region',layout=Layout(margin='20px 20px 20px 20px'))
container=widgets.HBox([region])

In [6]:
def plot_fig(id_r,title,x_axis,g,data,iterate_list):
    var={i.split()[0].lower():i for i  in regions}
    id_region=var[id_r]
    with g.batch_update():
        if len(iterate_list)!=0:
            for i in iterate_list:
                df = data[data['year'] == i]
                g.data[i-1957].y=df[id_region]
                g.data[i-1957].name=str(i)
                g.data[i-1957].x=df[x_axis]
        else:
            g.data[0].y=data[id_region]
            g.data[0].name=str(title)
            g.data[0].x=data[x_axis]

In [7]:
def initial_plot(x_axis,df,iterate_list,title,x_axis_title):
#     list_type=year
    traces=[]
    if len(iterate_list)==0:
        traces.append(go.Scatter(x=df[x_axis],y=df[regions[0]],name=regions[0]))
    else:
        for i in iterate_list:
            temp = df[df['year'] == i]
            traces.append(go.Scatter(x=temp[x_axis],y=temp[regions[0]],name=str(i)))
    layout = go.Layout(title=dict(text=title),xaxis=dict(title=x_axis_title,rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall (inch)'))
    go.Layout()
    return go.FigureWidget(data=traces, layout=layout)

In [8]:
def response(change):
    val=region.value.split()[0].lower()
    plot_fig(val,region.value,'day-number',g,rain_monsoon,years)
    plot_fig(val,region.value,'month',g_mean_monthly_year,mean_monthly_year,years)
    plot_fig(val,region.value,'month',g_median_monthly_year,median_monthly_year,years)
    plot_fig(val,region.value,'year',g_mean_yearly,mean_yearly,[])
    plot_fig(val,region.value,'year',g_median_yearly,median_yearly,[])
    plot_fig(val,region.value,'day-number',g_mean_daily,mean_daily,[])
    plot_fig(val,region.value,'day-number',g_median_daily,median_daily,[])
    plot_fig(val,region.value,'day-number',g_mean_daily_monsoon,mean_daily_monsoon,[])
    plot_fig(val,region.value,'day-number',g_median_daily_monsoon,median_daily_monsoon,[])

In [9]:
region.observe(response,names='value')

### Rain observed in all classification daily basIs (only monsoon time)

In [10]:
g=initial_plot('day-number',rain_monsoon,years,'Rain observed on daily basis (monsoon time)','Date of year')
total_container=widgets.VBox([container,g])
display(total_container)

### Rain Observed in all classification Monthly basis (mean and median)

In [11]:
mean_monthly_year=rain.groupby(['year','month'],as_index=False)[regions].mean()
median_monthly_year=rain.groupby(['year','month'],as_index=False)[regions].median()

In [12]:
g_mean_monthly_year=initial_plot('month',mean_monthly_year,years,'Mean rain observed on monthly basis','Month')
g_median_monthly_year=initial_plot('month',median_monthly_year,years,'Median rain observed on monthly basis','Month')
total_container=widgets.VBox([container,g_mean_monthly_year,g_median_monthly_year])
display(total_container)

### Rain Observed in all classification Yearly basis (mean and median)

In [13]:
mean_yearly=rain.groupby(['year'],as_index=False)[regions].mean()
median_yearly=rain.groupby(['year'],as_index=False)[regions].median()

In [14]:
g_mean_yearly=initial_plot('year',mean_yearly,[],'Mean rain observed on yearly basis','Year')
g_median_yearly=initial_plot('year',median_yearly,[],'Median rain observed on yearly basis','Year')
display(region,g_mean_yearly,g_median_yearly)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('ajmer', 'alwar', 'banswa…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': '455385b5-…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': 'a51578dc-…

### Rainfall day wise distribution in all classification :full year (mean and median)

In [15]:
mean_daily=rain.groupby(['day-number'],as_index=False)[regions].mean()
median_daily=rain.groupby(['day-number'],as_index=False)[regions].median()

In [16]:
g_mean_daily=initial_plot('day-number',mean_daily,[],'Mean distribution of Rainfall','Day of year')
g_median_daily=initial_plot('day-number',median_daily,[],'Median distribution of Rainfall','Day of year')
display(region,g_mean_daily,g_median_daily)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('ajmer', 'alwar', 'banswa…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': 'f7df5887-…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': 'bd6872e8-…

### Rainfall day wise distribution in all classification : monsoon time (mean and median)

In [17]:
mean_daily_monsoon=rain_monsoon.groupby(['day-number'],as_index=False)[regions].mean()
median_daily_monsoon=rain_monsoon.groupby(['day-number'],as_index=False)[regions].median()

In [18]:
g_mean_daily_monsoon=initial_plot('day-number',mean_daily_monsoon,[],'Mean distribution of Rainfall (monsoon time)','Day of year')
g_median_daily_monsoon=initial_plot('day-number',median_daily_monsoon,[],'Median distribution of Rainfall (monsoon time)','Day of year')
display(region,g_mean_daily_monsoon,g_median_daily_monsoon)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('ajmer', 'alwar', 'banswa…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': 'f6033bf0-…

FigureWidget({
    'data': [{'name': 'ajmer',
              'type': 'scatter',
              'uid': '1499995b-…

### Outlier calculation in all classification: year/month/daily

In [19]:
Ouitliers_monsoon_y=rain_monsoon.groupby(['year'],as_index=False)[regions].sum()

In [20]:
Ouitliers_monsoon_y.head(10)

,year,ajmer,alwar,banswara,baran,barmer,bharatpur,bhilwara,bikaner,bundi,...,bwhw,sub humid,humid,very humid,semi arid,arid,daw,caw,ead,dbw
0,1957,381.41,586.26,461.19,0.0,170.32,610.71,521.44,198.10,844.77,...,133.55,484.90,554.41,517.20,285.99,122.28,498.30,450.00,154.51,210.15
1,1958,345.54,758.34,834.39,0.0,158.20,1010.30,716.56,293.65,890.35,...,206.69,523.27,780.45,799.14,327.89,162.92,575.71,813.10,138.56,304.77
2,1959,236.43,700.82,501.49,0.0,202.29,664.27,483.34,191.40,583.26,...,188.80,428.24,539.10,493.38,269.48,137.29,439.35,518.19,133.14,259.18
3,1960,236.43,695.34,501.49,0.0,202.29,663.26,483.34,191.40,583.26,...,188.80,436.27,538.73,493.38,269.41,137.29,443.21,518.19,133.14,259.02
4,1961,0.00,565.70,1008.03,0.0,572.74,772.43,875.02,270.00,886.60,...,257.06,674.21,824.83,951.61,357.79,315.24,674.49,890.99,394.23,236.82
5,1962,296.61,475.28,862.72,0.0,171.19,604.64,487.00,222.59,678.14,...,195.34,416.40,568.34,885.58,289.38,192.30,437.72,678.37,186.26,286.66
6,1963,292.33,475.28,834.79,0.0,113.62,593.96,434.25,79.85,696.60,...,100.94,372.40,589.38,865.35,202.70,89.39,403.79,761.41,78.28,197.62
7,1964,392.92,475.28,507.94,0.0,293.50,720.23,557.02,240.55,497.81,...,261.36,542.83,549.18,500.48,390.75,217.91,529.47,468.50,193.11,392.76
8,1965,289.00,475.28,400.52,0.0,159.00,591.14,370.15,123.06,439.38,...,130.43,355.96,437.18,385.94,216.74,113.24,373.78,386.97,114.44,143.23
9,1966,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,680.73,592.54,0.00,151.70,0.00,608.05,678.90,0.00,275.40


In [21]:
Ouitliers_monsoon_y.isna()

,year,ajmer,alwar,banswara,baran,barmer,bharatpur,bhilwara,bikaner,bundi,...,bwhw,sub humid,humid,very humid,semi arid,arid,daw,caw,ead,dbw
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
q1, q2, q3= np.percentile(Ouitliers_monsoon_y,[25,50,75])

In [23]:
q1

198.60749999999996

In [24]:
q3

630.6999999999999

In [25]:
q2

415.7549999999999

In [26]:
q3-q1

432.0925

In [27]:
Ouitliers_monsoon_m=rain_monsoon.groupby(['month'],as_index=False)[regions].sum()

In [28]:
Ouitliers_monsoon_m

,month,ajmer,alwar,banswara,baran,barmer,bharatpur,bhilwara,bikaner,bundi,...,bwhw,sub humid,humid,very humid,semi arid,arid,daw,caw,ead,dbw
0,5,69.25,56.58,17.59,23.81,45.95,68.67,111.59,109.37,21.70,...,62.19,62.86,59.33,22.95,74.56,37.16,64.44,44.70,38.95,72.77
1,6,2379.09,3413.64,4878.69,2593.80,1155.24,2993.24,3248.71,1592.11,3741.16,...,1676.72,2975.32,3796.05,4953.25,2248.97,1476.51,3126.77,4673.23,1361.82,2118.15
2,7,8726.04,9906.62,15367.18,6553.93,3257.39,10585.87,11071.97,4227.62,12569.99,...,4008.32,10451.36,12328.68,14662.58,6752.84,3471.50,10747.55,13114.40,3235.46,5453.96
3,8,7939.99,10982.62,17735.30,6217.21,3836.33,12237.92,11337.31,2952.63,12365.22,...,3369.54,10067.86,12786.68,16167.03,5893.75,3074.67,10526.93,14416.95,3041.12,4844.49
4,9,2685.57,5467.43,7239.84,2840.66,1553.13,5776.67,4623.21,1532.20,5117.08,...,1640.33,4344.20,5617.65,6633.17,2547.47,1467.32,4513.18,6498.00,1366.72,2201.89
5,10,116.38,353.50,122.15,244.19,57.94,321.13,151.38,194.83,184.44,...,79.41,242.71,253.45,324.90,153.37,56.52,235.82,241.45,55.62,131.30


In [29]:
Ouitliers_monsoon_m.describe()

,month,ajmer,alwar,banswara,baran,barmer,bharatpur,bhilwara,bikaner,bundi,...,bwhw,sub humid,humid,very humid,semi arid,arid,daw,caw,ead,dbw
count,6.000000,6.000000,6.0000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,7.500000,3652.720000,5030.0650,7560.125000,3078.933333,1650.996667,5330.583333,5090.695000,1768.126667,5666.598333,...,1806.085000,4690.718333,5806.973333,7127.313333,2945.160000,1597.280000,4869.115000,6498.121667,1516.615000,2470.426667
std,1.870829,3795.358487,4660.9791,7536.448454,2813.754046,1595.339351,5172.501514,5051.645669,1597.844482,5629.503870,...,1634.687547,4612.226408,5642.794419,6931.266200,2823.823068,1451.535229,4780.829402,6174.233593,1388.738555,2278.338417
min,5.000000,69.250000,56.5800,17.590000,23.810000,45.950000,68.670000,111.590000,109.370000,21.700000,...,62.190000,62.860000,59.330000,22.950000,74.560000,37.160000,64.440000,44.700000,38.950000,72.770000
25%,6.250000,682.057500,1118.5350,1311.285000,831.592500,332.265000,989.157500,925.712500,529.172500,1073.620000,...,469.640000,925.862500,1139.100000,1481.987500,677.270000,409.220000,958.557500,1349.395000,382.170000,628.012500
50%,7.500000,2532.330000,4440.5350,6059.265000,2717.230000,1354.185000,4384.955000,3935.960000,1562.155000,4429.120000,...,1658.525000,3659.760000,4706.850000,5793.210000,2398.220000,1471.915000,3819.975000,5585.615000,1364.270000,2160.020000
75%,8.750000,6626.385000,8796.8225,13335.345000,5373.072500,2831.325000,9383.570000,9459.780000,2612.500000,10553.185000,...,2946.335000,8636.945000,10650.922500,12655.227500,5057.180000,2675.130000,9023.492500,11460.300000,2622.520000,4183.840000
max,10.000000,8726.040000,10982.6200,17735.300000,6553.930000,3836.330000,12237.920000,11337.310000,4227.620000,12569.990000,...,4008.320000,10451.360000,12786.680000,16167.030000,6752.840000,3471.500000,10747.550000,14416.950000,3235.460000,5453.960000


In [30]:
q1, q2, q3= np.percentile(Ouitliers_monsoon_m,[25,50,75])

In [31]:
q1

158.77

In [32]:
q2

2647.9

In [33]:
q3

5804.224999999995

In [34]:
q3-q1

5645.4549999999945